# PyKX Query Welcome Database
This notebook demonstrates querying the welcomedb database using PyKX.

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import boto3
import json
import datetime

import pykx as kx

from managed_kx import *
from env_kdb_1 import *

HDB_CLUSTER_NAME="cluster_welcomedb"

In [ ]:
# triggers credential get
session=None

try:
    # aws: use ada for credentials
    subprocess.call(["which", "ada"])
    os.system(f"ada credentials update --account={ACCOUNT_ID} --provider=isengard --role=Admin --once")
except: 
    None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS session: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

# Connect to Cluster

In [ ]:
# Query the HDB
hdb = get_pykx_connection(client, 
                          environmentId=ENV_ID, clusterName=HDB_CLUSTER_NAME, 
                          userName=KDB_USERNAME, boto_session=session)

# Table Schema

In [ ]:
schema_pdf = hdb("meta(`example)").pd()

display(schema_pdf)

# Query Table

In [ ]:
# Dates, counts, and analytics
hdb_pdf = hdb("select counts:count i, avg_num: avg number, avg_sq_num: avg sq number by date from example").pd()

# Number of Rows
hdb_rows = hdb("count example").py()

display(hdb_pdf)

# Number of Rows
print(f"Rows: {hdb_rows:,}")

In [ ]:
print( f"Last Run: {datetime.datetime.now()}" )